# Train Models and Choose The Best One!

### Import Basic Libraries

In [86]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

### Import The Data

In [87]:
data_path = r"data/cleaned_students.csv"
df = pd.read_csv(data_path)

In [88]:
df.head()

,gender,race ethnicity group,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,B,bachelor,standard,none,72,72,74
1,female,C,some_college,standard,completed,69,90,88
2,female,B,masters,standard,none,90,95,93
3,male,A,associates,free/reduced,none,47,57,44
4,male,C,some_college,standard,none,76,78,75


In [89]:
df.columns

Index(['gender', 'race ethnicity group', 'parental level of education',
       'lunch', 'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [90]:
# seperate the input feature and target 
X = df.drop(columns = 'math score')
y = df['math score']

In [91]:
X.head()

,gender,race ethnicity group,parental level of education,lunch,test preparation course,reading score,writing score
0,female,B,bachelor,standard,none,72,74
1,female,C,some_college,standard,completed,90,88
2,female,B,masters,standard,none,95,93
3,male,A,associates,free/reduced,none,57,44
4,male,C,some_college,standard,none,78,75


In [92]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

## Importing Models

In [93]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [94]:
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder , StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [95]:
print("Categories in 'gender' variable:     ", end = " " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  " , end = " ")
print(df['race ethnicity group'].unique())

print("Categories in'parental level of education' variable:" , end = " " )
print(df['parental level of education'].unique())

print("Categories in 'lunch' variable:     " , end = " " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     " , end = " " )
print(df['test preparation course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['B' 'C' 'A' 'D' 'E']
Categories in'parental level of education' variable: ['bachelor' 'some_college' 'masters' 'associates' 'high_school'
 'some_high_school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [96]:
X.head()

,gender,race ethnicity group,parental level of education,lunch,test preparation course,reading score,writing score
0,female,B,bachelor,standard,none,72,74
1,female,C,some_college,standard,completed,90,88
2,female,B,masters,standard,none,95,93
3,male,A,associates,free/reduced,none,57,44
4,male,C,some_college,standard,none,78,75


In [97]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [85]:
X.columns

Index(['gender', 'race ethnicity group', 'parental level of education',
       'lunch', 'test preparation course', 'reading score', 'writing score'],
      dtype='object')

## Make the Pipeline for the Models which need scaling